<a href="https://colab.research.google.com/github/eentartetekunst/NGS_bioinformatics/blob/main/NGS_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
! pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

In [ ]:
!conda install -y -c bioconda samtools

In [ ]:
! sudo apt install bedtools

In [ ]:
! conda install -c bioconda regtools

In [ ]:
! sudo apt-get update
! sudo apt-get -y install hisat2

In [ ]:
! conda install -c bioconda stringtie 

## download genome and its annotation

In [ ]:
! wget http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.chromosome.19.fa.gz #chr19 sequence
! wget http://ftp.ensembl.org/pub/release-107/gtf/mus_musculus/Mus_musculus.GRCm39.107.gtf.gz # annotation
! gzip -d Mus_musculus.GRCm39.dna.chromosome.19.fa.gz # unarchieve files

! touch chr19-annotation 
! gzip -d Mus_musculus.GRCm39.107.gtf.gz > chr19-annotation 

# select chr 19 from the annotation
! touch chr19-annotation 
! grep -P '^19\t' /content/Mus_musculus.GRCm39.107.gtf > chr19-annotation 

In [ ]:
! head -n2 chr19-annotation

19	ensembl	gene	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA";
19	ensembl	transcript	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; transcript_id "ENSMUST00000104035"; transcript_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA"; transcript_name "Gm22271-201"; transcript_source "ensembl"; transcript_biotype "snoRNA"; tag "basic"; transcript_support_level "NA (assigned to previous version 1)";


## map all reads on chr19 sequence (no trimming, known splice sites)



## extract coordinates of exons and splicing site from the annotation

In [ ]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_splice_sites.py
! chmod a+x extract_splice_sites.py
! ./extract_splice_sites.py 

In [ ]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_exons.py
! chmod a+x extract_exons.py
! ./extract_exons.py

In [ ]:
 # extract splicinge sites and exons from the annotation
! ./extract_splice_sites.py chr19-annotation > chr19.ss
! ! ./extract_exons.py chr19-annotation > chr19.exons 

# 1. Прокартируйте все образцы при помощи hisat2

In [ ]:
! hisat2-build --ss chr19.ss --exon chr19.exons Mus_musculus.GRCm39.dna.chromosome.19.fa base-index 

In [ ]:
%%shell 
reads_list='C14.5 B34 B14.5 C15.5 C17.5 B17.5 C34 B20 B15.5 C20' 
index='base-index'
fq='/content'

for i in ${reads_list}
do hisat2 --summary-file ${i}.log --no-softclip --no-unal -x ${index} -U ${fq}/${i}.fq.gz | samtools view -Sb - > ${i}.bam 
done

599461 reads; of these:
  599461 (100.00%) were unpaired; of these:
    49649 (8.28%) aligned 0 times
    538759 (89.87%) aligned exactly 1 time
    11053 (1.84%) aligned >1 times
91.72% overall alignment rate
599683 reads; of these:
  599683 (100.00%) were unpaired; of these:
    51939 (8.66%) aligned 0 times
    539715 (90.00%) aligned exactly 1 time
    8029 (1.34%) aligned >1 times
91.34% overall alignment rate
599672 reads; of these:
  599672 (100.00%) were unpaired; of these:
    47202 (7.87%) aligned 0 times
    540491 (90.13%) aligned exactly 1 time
    11979 (2.00%) aligned >1 times
92.13% overall alignment rate
600324 reads; of these:
  600324 (100.00%) were unpaired; of these:
    75959 (12.65%) aligned 0 times
    514263 (85.66%) aligned exactly 1 time
    10102 (1.68%) aligned >1 times
87.35% overall alignment rate
600268 reads; of these:
  600268 (100.00%) were unpaired; of these:
    59115 (9.85%) aligned 0 times
    531032 (88.47%) aligned exactly 1 time
    10121 (1.69

# 2. Соберите транскрипты при помощи stringtie для каждого образца используя аннотацию из ensembl (-G)

## for all of the samples

In [2]:
%%shell 
reads_list='C14.5 B34 B14.5 C15.5 C17.5 B17.5 C34 B20 B15.5 C20' 
# sort reads by their genomic location using samtools
for i in ${reads_list}
do samtools sort -o  ${i}.s.bam  ${i}.bam 
done

In [ ]:
# -G <ref.gtf> a reference annotation file used to guide the assembly process
# build gtf annotation of reads

%%shell 
reads_list='C14.5 B34 B14.5 C15.5 C17.5 B17.5 C34 B20 B15.5 C20' 

for i in ${reads_list}
do stringtie -o ${i}.annotation.gtf -G Mus_musculus.GRCm39.107.gtf ${i}.s.bam
done

In [4]:
! head B14.5.annotation.gtf

# stringtie -o B14.5.annotation.gtf -G Mus_musculus.GRCm39.107.gtf B14.5.s.bam
# StringTie version 2.2.1
19	StringTie	transcript	3133097	3137398	1000	+	.	gene_id "STRG.1"; transcript_id "STRG.1.1"; cov "2.314697"; FPKM "41.482002"; TPM "81.817490";
19	StringTie	exon	3133097	3133341	1000	+	.	gene_id "STRG.1"; transcript_id "STRG.1.1"; exon_number "1"; cov "2.361224";
19	StringTie	exon	3137331	3137398	1000	+	.	gene_id "STRG.1"; transcript_id "STRG.1.1"; exon_number "2"; cov "2.147059";
19	StringTie	transcript	3310774	3312089	1000	-	.	gene_id "STRG.2"; transcript_id "STRG.2.1"; cov "5.256909"; FPKM "94.209785"; TPM "185.815735";
19	StringTie	exon	3310774	3311635	1000	-	.	gene_id "STRG.2"; transcript_id "STRG.2.1"; exon_number "1"; cov "4.919954";
19	StringTie	exon	3311975	3312089	1000	-	.	gene_id "STRG.2"; transcript_id "STRG.2.1"; exon_number "2"; cov "7.782609";
19	StringTie	transcript	3314818	3332915	1000	-	.	gene_id "STRG.3"; transcript_id "STRG.3.1"; cov "5.645762"; FPKM "101.178474"

# 3. Перекартируйте риды используя новую аннотацию

Объединяем аннотации всех ридов

In [5]:
! touch mergelist.txt
! ls *.annotation.gtf > mergelist.txt
! cat mergelist.txt

B14.5.annotation.gtf
B15.5.annotation.gtf
B17.5.annotation.gtf
B20.annotation.gtf
B34.annotation.gtf
C14.5.annotation.gtf
C15.5.annotation.gtf
C17.5.annotation.gtf
C34.annotation.gtf


In [6]:
! stringtie --merge -G Mus_musculus.GRCm39.107.gtf -o stringtie_merged.gtf mergelist.txt # merge stringtie annotations for all of the samples

### Извлекаем координаты экзонов из новой аннотации

In [7]:
! head stringtie_merged.gtf

# stringtie --merge -G Mus_musculus.GRCm39.107.gtf -o stringtie_merged.gtf mergelist.txt
# StringTie version 2.2.1
19	StringTie	transcript	3103071	3247732	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3103071	3103278	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "1"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3109880	3109991	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "2"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3119367	3119440	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "3"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3122688	3122953	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "4"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTi

In [8]:
! cat stringtie_merged.gtf | awk ' $3 == "exon"' > stringtie.exon

In [10]:
! head stringtie.exon

19	StringTie	exon	3103071	3103278	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "1"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3109880	3109991	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "2"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3119367	3119440	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "3"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3122688	3122953	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "4"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3246945	3247029	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "5"; gene_name "1700030N03Rik"; ref_gene_id "ENSMUSG00000100969"; 
19	StringTie	exon	3247605	3247732	1000	-	.	gene_id "MSTRG.1"; transcript_id "ENSMUST00000237047"; exon_number "6"; 

### ^ это я вручную пыталась сделать то что делает функция ./extract_exons.py, теперь сделаем через нее:

In [16]:
# ! cat stringtie.exon | awk ' $3 == "exon" {print $1 " " $4 " " $5 " " $7}' > stringtie_coords.exon

In [17]:
! cat stringtie.exon | awk ' $3 == "exon" {print $1 "\t" $4 "\t" $5 "\t" $7}' > stringtie_coords.exon

In [18]:
! head stringtie_coords.exon

19	3103071	3103278	-
19	3109880	3109991	-
19	3119367	3119440	-
19	3122688	3122953	-
19	3246945	3247029	-
19	3247605	3247732	-
19	3115711	3118149	-
19	3119367	3119440	-
19	3122688	3122953	-
19	3246945	3247029	-


In [19]:
! head chr19.exons

19	3103070	3103277	-
19	3109879	3109990	-
19	3115710	3118148	-
19	3119366	3119439	-
19	3122687	3122952	-
19	3125713	3125884	-
19	3184644	3184958	+
19	3186983	3187089	-
19	3192759	3193064	+
19	3195867	3196264	-


### Мы получили координаты экзонов из новой аннотации, теперь надо сделать картирование с ними:

строим индекс с новымии координатами экзонов

In [23]:
! ./extract_splice_sites.py  stringtie_merged.gtf  >  stringtie_merged.ss
! ./extract_exons.py  stringtie_merged.gtf  >  stringtie_merged.exons 

# отбираем координаты для 19 хромасомы

In [28]:
! awk ' $1 == 19' stringtie_merged.ss > chr19_stringtie_merged.ss

In [29]:
! head chr19_stringtie_merged.ss

19	3103277	3109879	-
19	3109990	3119366	-
19	3118148	3119366	-
19	3119439	3122687	-
19	3122952	3125713	-
19	3122952	3246944	-
19	3125884	3246944	-
19	3133340	3137330	+
19	3196264	3197632	-
19	3196264	3203930	-


In [30]:
! awk ' $1 == 19' stringtie_merged.exons > chr19_stringtie_merged.exons

In [31]:
! head chr19_stringtie_merged.exons 

19	3103070	3103277	-
19	3109879	3109990	-
19	3115710	3118148	-
19	3119366	3119439	-
19	3122687	3122952	-
19	3125713	3125884	-
19	3133094	3133340	+
19	3137330	3137397	+
19	3184644	3184958	+
19	3186983	3187089	-


## строим индекс по новым координатам из аннотации и картируем риды

In [ ]:
! hisat2-build --ss  chr19_stringtie_merged.ss --exon chr19_stringtie_merged.exons  Mus_musculus.GRCm39.dna.chromosome.19.fa stringtie_index

In [33]:
%%shell 
reads_list='C14.5 B34 B14.5 C15.5 C17.5 B17.5 C34 B20 B15.5 C20' 
index='stringtie_index'
fq='/content'

for i in ${reads_list}
do hisat2 --summary-file ${i}.log --no-softclip --no-unal -x ${index} -U ${fq}/${i}.fq.gz | samtools view -Sb - > ${i}.bam 
done

599461 reads; of these:
  599461 (100.00%) were unpaired; of these:
    49525 (8.26%) aligned 0 times
    538680 (89.86%) aligned exactly 1 time
    11256 (1.88%) aligned >1 times
91.74% overall alignment rate
599683 reads; of these:
  599683 (100.00%) were unpaired; of these:
    51983 (8.67%) aligned 0 times
    539387 (89.95%) aligned exactly 1 time
    8313 (1.39%) aligned >1 times
91.33% overall alignment rate
599672 reads; of these:
  599672 (100.00%) were unpaired; of these:
    46948 (7.83%) aligned 0 times
    540589 (90.15%) aligned exactly 1 time
    12135 (2.02%) aligned >1 times
92.17% overall alignment rate
600324 reads; of these:
  600324 (100.00%) were unpaired; of these:
    75829 (12.63%) aligned 0 times
    514122 (85.64%) aligned exactly 1 time
    10373 (1.73%) aligned >1 times
87.37% overall alignment rate
600268 reads; of these:
  600268 (100.00%) were unpaired; of these:
    59069 (9.84%) aligned 0 times
    530897 (88.44%) aligned exactly 1 time
    10302 (1.72

In [ ]:
#! mkdir annotations
#! mkdir reads
#! mkdir alignments 
#! mkdir indexes

! mv *.ht2 /content/indexes
! mv *.log *.gz  /content/reads 
! mv *.gtf *.exons *.exon *.ss /content/annotations
! mv *.bam  /content/alignments

In [50]:
! ls

alignments		 Mus_musculus.GRCm39.dna.chromosome.19.fa
annotations		 reads
chr19-annotation	 sample_data
extract_exons.py	 stringtie_coords.exon
extract_splice_sites.py  stringtie.exon
indexes			 tmp_im02Ex
mergelist.txt		 tmp_vbCcqH
